In [52]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
#불러오기
columns = ['media','date','title','article_original','url']
news_crawled_df = pd.read_csv('./crawled_news/news_df_210205_v02.csv')
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
866,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
867,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
868,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
869,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
news_crawled_df.isna().sum()

media                0
date                 0
title                0
article_original    13
url                  0
dtype: int64

In [4]:
news_crawled_df.dropna(inplace=True)
news_crawled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 858 entries, 0 to 870
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   media             858 non-null    object
 1   date              858 non-null    object
 2   title             858 non-null    object
 3   article_original  858 non-null    object
 4   url               858 non-null    object
dtypes: object(5)
memory usage: 40.2+ KB


In [5]:
news_crawled_df.reset_index(inplace=True)
news_crawled_df.drop('index',inplace=True,axis=1)
news_crawled_df

,media,date,title,article_original,url
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [6]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None)
stopwords = list(stopwords2[0])
stopwords

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [7]:
#형태소 토크나이징
from konlpy.tag import Okt
okt = Okt()

articles = news_crawled_df.article_original

articles_morphed = []
for article in articles:
    morph = okt.morphs(article)
    
    stopped = ''
    for word in morph:
        if word not in stopwords:
            stopped += word
            stopped += ' '
            
    articles_morphed.append(stopped)

In [8]:
#형태소 분해된 컬럼 추가
news_crawled_df['article_morphed'] = articles_morphed
#날짜 칼럼 추가
news_crawled_df['date_pd']=pd.to_datetime(news_crawled_df['date'])
news_crawled_df

,media,date,title,article_original,url,article_morphed,date_pd
0,조선일보,2020-01-02,150만명 몰린 타임스스퀘어 새해 무대도 BTS,사회자의 소개와 함께 방탄소년단BTS이 모습을 드러냈다. 2019년의 마지막 날 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,사회자 소개 방탄소년단 BTS 모습 드러냈다 2019년 마지막 날 미국 뉴욕 맨해튼...,2020-01-02
1,조선일보,2020-01-02,방탄소년단 CNN 선정 2010년대 음악 변화시킨 아티스트,12월 31일현지시각 CNN은 자신들의 음악 장르는 물론 스스로를 재창조하고 인상...,https://news.naver.com/main/read.nhn?mode=LSD&...,12월 31일 현지시각 CNN 음악 장르 스스로 재창 조 하고 인상 적 인 영향력 ...,2020-01-02
2,조선일보,2020-01-03,방탄소년단 새앨범 작업중 새해도 K팝 인베이전 이어진다,경자년庚子年 새해에도 K팝 스타들의 세계 무대 공략은 계속될 전망이다. 특히 지난...,https://news.naver.com/main/read.nhn?mode=LSD&...,경자년 庚子年 새해 에도 K 팝 스타 세계 무대 공략 계속 될 전망 특히 지난해 전...,2020-01-03
3,조선일보,2020-01-03,서울시 BTS트와이스 활용해 K팝 관광명소 추천,K팝에 대한 관심이 전 세계적으로 높아지면서 서울시가 이른바 ‘K팝 명소’들을 선정...,https://news.naver.com/main/read.nhn?mode=LSD&...,K 팝 대한 관심 전 세계 적 높아지면서 서울시 이른바 ‘ K 팝 명소 ’ 들을 선...,2020-01-03
4,조선일보,2020-01-06,봉준호 BTS 영향력은 나의 3천배 멋진 아티스트의 나라,골든글로브 시상식에서 한국 최초로 외국어영화상을 수상한 기생충의 봉준호 감독이 한 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,골든글로브 시상식 한국 최초 외국어 영화상 수상한 기생충 봉준호 감독 한 매체 인터...,2020-01-06
...,...,...,...,...,...,...,...
853,한겨레,2020-12-26,누구도 가지 못한 길을 여는 BTS라는 세계,코로나19의 대유행으로 고통받은 한해 방탄소년단BTS은 큰 위로가 되어주었다. 20...,https://news.naver.com/main/read.nhn?mode=LSD&...,코로나 19 대유행 고통 받은 한해 방탄소년단 BTS 큰 위로 되어주었다 2013년...,2020-12-26
854,한겨레,2020-12-26,세계의 모든 힘없는 자들에게 2020 BTS 혁명,“이 세계의 모든 힘없는 자들아 우리가 패배할 날이 올지도 모르지만 오늘은 아니야 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,“ 세계 모든 힘없는 패배할 날 올 지도 모르지만 오늘 아니야 오늘 싸워 ”‘ 낫 ...,2020-12-26
855,조선일보,2020-12-29,코로나로 힘든 1년 트롯맨봉준호BTS 있어 웃고 울었다,2020년은 한국 문화계의 저력과 바닥이 모두 드러난 해였다. 전례 없던 세계 기...,https://news.naver.com/main/read.nhn?mode=LSD&...,2020년 한국 문화 계 저력 바닥 드러난 해였다 전례 없던 세계 기록 약속 이나 ...,2020-12-29
856,중앙일보,2020-12-30,박영선 삶은 아무 일 없단 듯 계속된다BTS 노래로 신년사,박영선 중소벤처기업부 장관은 30일 발표한 신년사에서 “이미 최선을 다 하고 계신데...,https://news.naver.com/main/read.nhn?mode=LSD&...,박영선 중소 벤처기업 부 장관 30일 발표 한 신년사 “ 이미 최선 다 하고 계신데...,2020-12-30


In [9]:
news_crawled_df.loc[69]

media                                                            중앙일보
date                                                       2020-02-12
title                               이해찬 제2의 기생충BTS 위해 노력지원하되 간섭하지 않겠다
article_original    이해찬 더불어민주당 대표는 12일 “앞으로도 제2의 제3의 ‘기생충’ 같은 영화와 ...
url                 https://news.naver.com/main/read.nhn?mode=LSD&...
article_morphed     이해찬 민주당 대표 12일 “ 앞 으로도 2 3 ‘ 기생충 ’ 같은 영화 방탄소년단...
date_pd                                           2020-02-12 00:00:00
Name: 69, dtype: object

In [49]:
#기사 날짜 14일마다 그룹화
date_grouped = []
date_range = pd.date_range(start='2020-01-01', end='2020-12-31',periods=26)
L = len(date_range)
R = len(news_crawled_df)

for i, date in enumerate(date_range):
    index_group = []
    if i< L-1:
        for j in range(R):
            if date_range[i] < news_crawled_df.date_pd[j] <= date_range[i+1]:
                index_group.append(j)
        date_grouped.append(index_group)
    
print(date_grouped)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22], [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], [54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130], [131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156], [157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173], [174, 175, 176, 177, 178, 179, 180], [181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], [194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211], [212, 213, 214, 215, 216, 217, 

In [11]:
news_crawled_df.date_pd[857]

Timestamp('2020-12-30 00:00:00')

In [12]:
len(date_grouped[0])

23

In [51]:
# 그룹별 tfidf 변환
def tfidfy(period_no):
    tfidf = TfidfVectorizer(lowercase=False)
    #그룹에 있는 아티클 가져오기
    article_group = [news_crawled_df.article_original[i] for i in date_grouped[period_no]]

    art_group_tfidf = tfidf.fit_transform(article_group)
    return art_group_tfidf


<59x6337 sparse matrix of type '<class 'numpy.float64'>'
	with 12189 stored elements in Compressed Sparse Row format>

In [46]:
#date 그룹 내에서 유사도 높은 기사의 index 가져오는 함수
def get_similar_art_index(article_index):
    # 그룹 안의 첫 번째 기사와 나머지 기사간의 similarity 비교
    sim_pair = cosine_similarity(art_group_tfidf[article_index],art_group_tfidf)
    # 인덱스 내림차순으로 소트
    sorted_index = sim_pair.argsort()[:,::-1]
    # 자기자신은 빼기
    sorted_index = sorted_index[:,1:]
    print(sorted_index)

    # #유사도가 큰 순으로 group을 추출해 재정렬
    # group = pd.Series(date_grouped[0])
    # art_sorted_indices = group[sorted_index.reshape(-1)]
    # print(art_sorted_indices)

    #유사도가 큰 순으로 유사도 값을 재정렬하되 자기 자신은 제외
    art_sim_value = np.sort(sim_pair.reshape(-1))[::-1]
    art_sim_value = art_sim_value[1:]
    print(art_sim_value)

    # 유사도가 0.3보다 높은 인덱스만 뽑아오기.
    index_high = []
    for num,value in enumerate(art_sim_value):
        if value>=0.3:
            index_high.append(sorted_index[:,num].tolist())
            
    return index_high

In [47]:
#psuedocode
# 
tfidfy(6)
get_similar_art_index(6)

for 

[[ 7 10  0  8  9 18 17 21 22  2 20  3  1 12 13 19  5 15 14 16 11  4]]
[0.48176257 0.08662749 0.07715081 0.06506406 0.05393581 0.04741171
 0.04690826 0.03682747 0.0353215  0.02743286 0.02573512 0.02304317
 0.01945191 0.01834062 0.01739335 0.01715704 0.01618937 0.01205435
 0.01149324 0.0091562  0.00809762 0.00453107]
[[7]]
